# Scrape LinkedIn Using Selenium, Request and Beautiful Soup in Python

We are going to scrape Linkedin Jobs. More specifically, the following details will be scraped:
- Job Id
- Job title
- Seniority Level
- Location
- Job description
- number of candidats
- posted time ago

1. To scrape Job Ids, we will use `selenium` to navigate to this URL: `https://www.linkedin.com/jobs/search?`.

`chromedriver` executable and your LinkedIn credentials are required here.

3. As explained [here](https://www.scrapingdog.com/blog/scrape-linkedin-jobs/), to scrape other details (level, description...), we will use a simple GET request (leveraging the `requests` library) to this URL: `https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/xxxxxx` where xxxxxx is the job ID.\
This is easier than using clicks from `selenium`.

Note that I tried to scrape Job IDs using the guest URL `https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?` but the results were imprecise.

# I-Scraping Linkedin Jobs IDs using selenium and BeautifulSoup

In [1]:
# pip install selenium 
# pip install beautifulsoup4

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests

import time, datetime
import pandas as pd
import numpy as np
import math, re, sys
import warnings
warnings.filterwarnings("ignore")

## Login to Linkedin using selenium

🔑 **Note:**

1. To use `selenium`, we need a web driver. For instance, the `chromedriver` can be downloaded from [here](https://chromedriver.chromium.org/downloads).\
   Next, we add the chromedriver to the project directory.

3. Linkedin credentials (email address and password) are also required. You can save them here: `../data/user_credentials.txt`

In [80]:
# Get User Credentials
with open('/Users/Jason/Desktop/is3107/is3107_DataEngineering/user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
    
my_email,my_pwd = user_credentials[0],user_credentials[1]
my_email,my_pwd

('bt4222project@gmail.com', 'bt4222project')

In [81]:
# 1. Instanciate the chrome service
chromedriver_path = "C:/Users/Jason/Desktop/IS3107/chromedriver.exe"
service = Service(executable_path=chromedriver_path)

# 2. Instanciate the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options, service=service)

# 3. Open the LinkedIn login page
driver.get('https://www.linkedin.com/login')
time.sleep(10) # waiting for the page to load

# 4. Enter email address & password
email_input = driver.find_element(By.ID, 'username')
password_input = driver.find_element(By.ID, 'password')
email_input.send_keys(my_email)
password_input.send_keys(my_pwd)

# 5. Click the login button
password_input.send_keys(Keys.ENTER)

time.sleep(10)

We will be logged into LinkedIn after running the above code.

## Scraping Linkedin Jobs IDs

1. Set the search query parameters: keywords (ie. Job title) and location;
2. Search results are displayed on many pages: `25` jobs are listed on each page;
3. We will navigate to every page using the `start` parameter (0,25,50...);
4. We need to scroll to the bottom of the page to load the full data;
5. To get Job Ids, we will parse the HTML content of the page using BeautifulSoup.

In [83]:
List_Job_IDs = []

In [84]:
# Create a function 'Scroll to the bottom'. 

# time.sleep() function is used to provide extra time for the webpage to load. 
# I used 120 seconds. If the 25 jobs have not loaded during this period, we can make adjust it and test again.

def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(sleep_time)  

In [85]:
# Navigate to the first page (start=0) and scroll to the bottom of the page

keywords = 'software%20engineer'
location = 'singapore'
start = 0

url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
url = requests.utils.requote_uri(url)
driver.get(url)
scroll_to_bottom(driver,sleep_time=10)

In [86]:
# Get number of jobs found and number of pages:

# Parse the HTML content of the page using BeautifulSoup.
soup = BeautifulSoup(driver.page_source, 'html.parser')

try:
    div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
    number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
except:
    number_of_jobs = 0
    
number_of_pages=math.ceil(number_of_jobs/25)
print("number_of_jobs:",number_of_jobs)
print("number_of_pages:",number_of_pages)

number_of_jobs: 0
number_of_pages: 0


In [67]:
# Get Job Ids present on the first page.

def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
        # job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
        # location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    
    return Job_Ids_on_the_page    

Jobs_on_this_page = find_Job_Ids(soup)
List_Job_IDs.extend(Jobs_on_this_page)

Now that we've scraped the job IDs and number of results from the first page, let's iterate over the remaining pages.

### Iterate over the remaining pages

In [68]:
if number_of_pages>1:
    
    for page_num in range(1,number_of_pages):
        print(f"Scraping page: {page_num}",end="...")
        
        # Navigate to page
        url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={25 * page_num}'
        url = requests.utils.requote_uri(url)
        driver.get(url)
        scroll_to_bottom(driver)

        # Parse the HTML content of the page using BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Get Job Ids present on the page.
        Jobs_on_this_page = find_Job_Ids(soup)
        List_Job_IDs.extend(Jobs_on_this_page)  
        print(f'Jobs found:{len(Jobs_on_this_page)}')

pd.DataFrame({"Job_Id":List_Job_IDs}).to_csv('Job_Ids.csv',index=False)

In [69]:
## Close the browser and shut down the ChromiumDriver executable that
# is started when starting the ChromiumDriver. 
driver.quit()

## Scraping Job description using requests and BeautifulSoup
https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

In [70]:
import requests
from bs4 import BeautifulSoup

list_job_IDs = pd.read_csv("Job_Ids.csv").Job_Id.to_list()

In [71]:
list_job_IDs = list_job_IDs
list_job_IDs

[3879301587,
 3820479502,
 3907751399,
 3771637074,
 3900798064,
 3849949342,
 3887992941,
 3888577120,
 3893935374,
 3903524965,
 3902451922,
 3888057008,
 3789170493,
 3900502509,
 3866964302,
 3828523024,
 3879301571,
 3780614897,
 3829053920,
 3882844672,
 3835903406,
 3814291603,
 3811232439,
 3883821374,
 3870146673]

In [72]:
def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [73]:
job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
job={}
list_jobs=[]

for j in range(0,len(list_job_IDs)):
    print(f"{j+1} ... read jobId:{list_job_IDs[j]}")

    resp = requests.get(job_url.format(list_job_IDs[j]))
    soup=BeautifulSoup(resp.text,'html.parser')
    # print(soup.prettify()) 

    job["Job_ID"] = list_job_IDs[j] 
    # try:
    #     job["Job_html"] = resp.content
    # except:
    #     job["Job_html"]=None

    try: # remove tags
        job["Job_txt"] = remove_tags(resp.content)
    except:
        job["Job_txt"] = None
    
    try:
        job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        job["company"]=None

    try:
        job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        job["job-title"]=None

    try:
        job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        job["level"]=None

    try:
        job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job["location"]=None

    try:
        job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job["posted-time-ago"]=None

    try:
        nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
        nb_candidats = int(nb_candidats.split()[0])
        job["nb_candidats"]= nb_candidats
    except:
        job["nb_candidats"]=None

    list_jobs.append(job)
    job={}

# create a pandas Datadrame
jobs_DF = pd.DataFrame(list_jobs)

1 ... read jobId:3879301587


2 ... read jobId:3820479502
3 ... read jobId:3907751399
4 ... read jobId:3771637074
5 ... read jobId:3900798064
6 ... read jobId:3849949342
7 ... read jobId:3887992941
8 ... read jobId:3888577120
9 ... read jobId:3893935374
10 ... read jobId:3903524965
11 ... read jobId:3902451922
12 ... read jobId:3888057008
13 ... read jobId:3789170493
14 ... read jobId:3900502509
15 ... read jobId:3866964302
16 ... read jobId:3828523024
17 ... read jobId:3879301571
18 ... read jobId:3780614897
19 ... read jobId:3829053920
20 ... read jobId:3882844672
21 ... read jobId:3835903406
22 ... read jobId:3814291603
23 ... read jobId:3811232439
24 ... read jobId:3883821374
25 ... read jobId:3870146673


In [74]:
jobs_DF.head()

,Job_ID,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats
0,3879301587,"Full Stack Engineer e2i, Employment & Employab...","e2i, Employment & Employability Institute",Full Stack Engineer,Executive,"Singapore, Singapore",2 weeks ago,185.0
1,3820479502,,None,None,None,None,None,NaN
2,3907751399,,None,None,None,None,None,NaN
3,3771637074,"Software Engineer Intern, Business Infrastruct...",TikTok,"Software Engineer Intern, Business Infrastruct...",Not Applicable,Singapore,2 months ago,177.0
4,3900798064,,None,None,None,None,None,NaN


🔑 **Note:**

Now we have scraped all Linkedin Job details. 

The next step is to process the data:
1. Create a posted_date column using posted_time_ago;
2. Clean up 'Job_description' (remove sentences like "Remove photo First name Last name Email Password ( 8 + characters )");
3. Clean up the 'level' column.

## Process data

In [75]:
def clean_Job_description(text):
    senetences_to_remove = ["Remove photo First name Last name Email Password (8+ characters) ",
                            "By clicking Agree & Join",
                            "you agree to the LinkedIn User Agreement",
                            "Privacy Policy and Cookie Policy",
                            "Continue Agree & Join or Apply on company website",
                            "Security verification",
                            "Close Already on LinkedIn ?",
                            "Close Already on LinkedIn?",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile , or create a new one",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile, or create a new one",
                            "Your job seeking activity is only visible to you",
                            "Email Continue Welcome back"]
    for sentence in senetences_to_remove:
        result = text.find(sentence)
        if result>-1:
            text = text[:result] + text[result+len(sentence):] # remove sentence from text

    return text 

In [76]:
def get_posted_date(posted_time_ago,date_scraping):
    """Convert posted_time_ago to number of days.
    For example, 1 month ago is replaced by 30. 1 week by 7 and so on..."""
    posted_date = None
    
    try:
        details = posted_time_ago.split()
        N_DAYS_AGO = int(details[0])
        day_week_month_year = details[1] 
        if day_week_month_year.startswith("day"):
            N_DAYS_AGO = N_DAYS_AGO
        elif day_week_month_year.startswith("week"):
            N_DAYS_AGO = N_DAYS_AGO*7
        elif day_week_month_year.startswith("month"):
            N_DAYS_AGO = N_DAYS_AGO*30
        elif day_week_month_year.startswith("year"):
            N_DAYS_AGO = N_DAYS_AGO*365
        else:
            N_DAYS_AGO = None

        posted_date = date_scraping - datetime.timedelta(days=N_DAYS_AGO)
    except:
        posted_date = None

    return posted_date

In [77]:
jobs_DF['scraping_date'] = pd.to_datetime(datetime.date.today())
jobs_DF['posted_date'] = np.vectorize(get_posted_date)(jobs_DF['posted-time-ago'], jobs_DF['scraping_date'])

jobs_DF['Job_txt'] = jobs_DF['Job_txt'].apply(clean_Job_description)
jobs_DF.level = jobs_DF.level.apply(lambda x:x.replace("Employment type\n        \n\n          ","") if x is not None else x)

jobs_DF.head()

,Job_ID,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats,scraping_date,posted_date
0,3879301587,"Full Stack Engineer e2i, Employment & Employab...","e2i, Employment & Employability Institute",Full Stack Engineer,Executive,"Singapore, Singapore",2 weeks ago,185.0,2024-04-22,2024-04-08
1,3820479502,,None,None,None,None,None,NaN,2024-04-22,NaT
2,3907751399,,None,None,None,None,None,NaN,2024-04-22,NaT
3,3771637074,"Software Engineer Intern, Business Infrastruct...",TikTok,"Software Engineer Intern, Business Infrastruct...",Not Applicable,Singapore,2 months ago,177.0,2024-04-22,2024-02-22
4,3900798064,,None,None,None,None,None,NaN,2024-04-22,NaT


## Save to json file

In [78]:
jobs_DF.to_json("softwareDeveloper_scraped.json")
jobs_DF.to_csv("softwareDeveloper_scraped.csv")

In [17]:
def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(sleep_time)  
    
def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
        # job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
        # location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    
    return Job_Ids_on_the_page 

def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

def clean_Job_description(text):
    senetences_to_remove = ["Remove photo First name Last name Email Password (8+ characters) ",
                            "By clicking Agree & Join",
                            "you agree to the LinkedIn User Agreement",
                            "Privacy Policy and Cookie Policy",
                            "Continue Agree & Join or Apply on company website",
                            "Security verification",
                            "Close Already on LinkedIn ?",
                            "Close Already on LinkedIn?",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile , or create a new one",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile, or create a new one",
                            "Your job seeking activity is only visible to you",
                            "Email Continue Welcome back"]
    for sentence in senetences_to_remove:
        result = text.find(sentence)
        if result>-1:
            text = text[:result] + text[result+len(sentence):] # remove sentence from text

    return text 

def get_posted_date(posted_time_ago,date_scraping):
    """Convert posted_time_ago to number of days.
    For example, 1 month ago is replaced by 30. 1 week by 7 and so on..."""
    posted_date = None
    
    try:
        details = posted_time_ago.split()
        N_DAYS_AGO = int(details[0])
        day_week_month_year = details[1] 
        if day_week_month_year.startswith("day"):
            N_DAYS_AGO = N_DAYS_AGO
        elif day_week_month_year.startswith("week"):
            N_DAYS_AGO = N_DAYS_AGO*7
        elif day_week_month_year.startswith("month"):
            N_DAYS_AGO = N_DAYS_AGO*30
        elif day_week_month_year.startswith("year"):
            N_DAYS_AGO = N_DAYS_AGO*365
        else:
            N_DAYS_AGO = None

        posted_date = date_scraping - datetime.timedelta(days=N_DAYS_AGO)
    except:
        posted_date = None

    return posted_date

In [18]:
def linkedin_scrapper(keyword='AI%20developer', user_credentials_path='/Users/Jason/Desktop/is3107/is3107_DataEngineering/user_credentials.txt', chromedriver_path="C:/Users/Jason/Desktop/IS3107/chromedriver.exe"):
    # Get User Credentials
    with open(user_credentials_path, 'r',encoding="utf-8") as file:
        user_credentials = file.readlines()
        user_credentials = [line.rstrip() for line in user_credentials]
    my_email,my_pwd = user_credentials[0],user_credentials[1]

    # 1. Instanciate the chrome service
    chromedriver_path = chromedriver_path
    service = Service(executable_path=chromedriver_path)

    # 2. Instanciate the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options, service=service)

    # 3. Open the LinkedIn login page
    driver.get('https://www.linkedin.com/login')
    time.sleep(10) # waiting for the page to load

    # 4. Enter email address & password
    email_input = driver.find_element(By.ID, 'username')
    password_input = driver.find_element(By.ID, 'password')
    email_input.send_keys(my_email)
    password_input.send_keys(my_pwd)

    # 5. Click the login button
    password_input.send_keys(Keys.ENTER)
    time.sleep(60)

    List_Job_IDs = []
    keywords = keyword
    location = 'singapore'
    start = 0
    url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
    url = requests.utils.requote_uri(url)
    driver.get(url)
    scroll_to_bottom(driver,sleep_time=10)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
        number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
    except:
        number_of_jobs = 0
    number_of_pages=math.ceil(number_of_jobs/25)
    print("number_of_jobs:",number_of_jobs)
    print("number_of_pages:",number_of_pages)

    # Get Job Ids present on the first page.
    Jobs_on_this_page = find_Job_Ids(soup)
    List_Job_IDs.extend(Jobs_on_this_page)

    if number_of_pages>1:
        for page_num in range(1,number_of_pages):
            print(f"Scraping page: {page_num}",end="...")
            
            # Navigate to page
            url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={25 * page_num}'
            url = requests.utils.requote_uri(url)
            driver.get(url)
            scroll_to_bottom(driver)
            
            # Parse the HTML content of the page using BeautifulSoup.
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Get Job Ids present on the page.
            Jobs_on_this_page = find_Job_Ids(soup)
            List_Job_IDs.extend(Jobs_on_this_page)  
            print(f'Jobs found:{len(Jobs_on_this_page)}')

    ## Close the browser and shut down the ChromiumDriver executable that
    # is started when starting the ChromiumDriver. 
    driver.quit()

    job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
    job={}
    list_jobs=[]

    for j in range(0,len(List_Job_IDs)):
        print(f"{j+1} ... read jobId:{List_Job_IDs[j]}")

        resp = requests.get(job_url.format(List_Job_IDs[j]))
        soup=BeautifulSoup(resp.content,'html.parser')

        job["Job_ID"] = List_Job_IDs[j]

        try:
            # apply_link = soup.find("a", {'class':"jobs-apply-button"})["href"]
            apply_link = 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId={}'.format(List_Job_IDs[j])
            # apply_link = job_url.format(List_Job_IDs[j])
            job['url'] = apply_link
        except:
            job['url'] = None

        try: # remove tags
            job["Job_txt"] = remove_tags(resp.content)
            # job["Job_txt"] = soup.find("div", {"class":"description__text description__text--rich"}).text.strip()
        except:
            job["Job_txt"] = None
            
        try:
            job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
        except:
            job["company"]=None
            
        try:
            job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
        except:
            job["job-title"]=None
            
        try:
            job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
        except:
            job["level"]=None
        
        try:
            job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
        except:
            job["location"]=None
            
        try:
            job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job["posted-time-ago"]=None
            
        try:
            nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
            nb_candidats = int(nb_candidats.split()[0])
            job["nb_candidats"]= nb_candidats
        except:
            job["nb_candidats"]=None
            
        list_jobs.append(job)
        job={}   

    # create a pandas Datadrame
    jobs_DF = pd.DataFrame(list_jobs) 

    jobs_DF['scraping_date'] = pd.to_datetime(datetime.date.today())
    jobs_DF['posted_date'] = np.vectorize(get_posted_date)(jobs_DF['posted-time-ago'], jobs_DF['scraping_date'])

    jobs_DF['Job_txt'] = jobs_DF['Job_txt'].apply(clean_Job_description)
    jobs_DF.level = jobs_DF.level.apply(lambda x:x.replace("Employment type\n        \n\n          ","") if x is not None else x)

    jobs_DF.to_json("{}_scraped.json".format(keyword.replace('%20', '')))
    jobs_DF.to_csv("{}_scraped.csv".format(keyword.replace('%20', '')))
    return jobs_DF

In [19]:
linkedin_scrapper('software%20engineer')
linkedin_scrapper('data%20analyst')
linkedin_scrapper('data%20scientist')
linkedin_scrapper('machine%20learning%20engineer')
linkedin_scrapper('AI%20developer')
linkedin_scrapper('data%20engineer')

number_of_jobs: 0
number_of_pages: 0
1 ... read jobId:3879301587
2 ... read jobId:3820479502
3 ... read jobId:3876416396
4 ... read jobId:3890442134
5 ... read jobId:3780614897
6 ... read jobId:3771637074
7 ... read jobId:3900798064
8 ... read jobId:3893119219
9 ... read jobId:3843041729
10 ... read jobId:3882844672
11 ... read jobId:3888577120
12 ... read jobId:3879301571
13 ... read jobId:3903524965
14 ... read jobId:3829053920
15 ... read jobId:3900502509
16 ... read jobId:3849949342
17 ... read jobId:3775187042
18 ... read jobId:3688875998
19 ... read jobId:3783567440
20 ... read jobId:3751241217
21 ... read jobId:3888052027
22 ... read jobId:3883629706
23 ... read jobId:3794624979
24 ... read jobId:3814291603
25 ... read jobId:3897134160


,Job_ID,url,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats,scraping_date,posted_date
0,3879301587,https://www.linkedin.com/jobs/collections/reco...,"Full Stack Engineer e2i, Employment & Employab...","e2i, Employment & Employability Institute",Full Stack Engineer,Executive,"Singapore, Singapore",3 weeks ago,191.0,2024-04-23,2024-04-02
1,3820479502,https://www.linkedin.com/jobs/collections/reco...,,None,None,None,None,None,NaN,2024-04-23,NaT
2,3876416396,https://www.linkedin.com/jobs/collections/reco...,IT Infrastructure Engineer Resorts World Sento...,Resorts World Sentosa,IT Infrastructure Engineer,Mid-Senior level,"Singapore, Singapore",3 weeks ago,199.0,2024-04-23,2024-04-02
3,3890442134,https://www.linkedin.com/jobs/collections/reco...,,None,None,None,None,None,NaN,2024-04-23,NaT
4,3780614897,https://www.linkedin.com/jobs/collections/reco...,Senior IT Infrastructure Engineer ONE Singapor...,ONE,Senior IT Infrastructure Engineer,Mid-Senior level,"Singapore, Singapore",2 weeks ago,NaN,2024-04-23,2024-04-09
5,3771637074,https://www.linkedin.com/jobs/collections/reco...,"Software Engineer Intern, Business Infrastruct...",TikTok,"Software Engineer Intern, Business Infrastruct...",Not Applicable,Singapore,2 months ago,177.0,2024-04-23,2024-02-23
6,3900798064,https://www.linkedin.com/jobs/collections/reco...,Internship - Software Engineer Infineon Techno...,Infineon Technologies,Internship - Software Engineer,Entry level,Singapore,1 week ago,114.0,2024-04-23,2024-04-16
7,3893119219,https://www.linkedin.com/jobs/collections/reco...,,None,None,None,None,None,NaN,2024-04-23,NaT
8,3843041729,https://www.linkedin.com/jobs/collections/reco...,"Software Engineer Intern Thales Singapore, Sin...",Thales,Software Engineer Intern,Not Applicable,"Singapore, Singapore",1 month ago,NaN,2024-04-23,2024-03-24
9,3882844672,https://www.linkedin.com/jobs/collections/reco...,"IT Systems Engineer Crone Corkill Singapore, S...",Crone Corkill,IT Systems Engineer,Mid-Senior level,"Singapore, Singapore",2 weeks ago,NaN,2024-04-23,2024-04-09
